In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/lite"

/content/drive/.shortcut-targets-by-id/156a1XWT2xtka_4scjTAqx3Fn2Te1P_OB/lite


In [ ]:
!git clone https://github.com/luka-group/lite

Cloning into 'lite'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 62 (delta 28), reused 34 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 180.55 KiB | 994.00 KiB/s, done.
Resolving deltas: 100% (28/28), done.


In [ ]:
%cd /content/drive/MyDrive/lite/lite

/content/drive/.shortcut-targets-by-id/156a1XWT2xtka_4scjTAqx3Fn2Te1P_OB/lite/lite


In [ ]:
!ls

data	    eval.py  LICENSE  lite.sh	output	     readme	result.py
dataset.py  eval.sh  lite.py  model.py	__pycache__  README.md	result.sh


In [ ]:
%cd /content/drive/MyDrive/lite/lite/data

/content/drive/.shortcut-targets-by-id/156a1XWT2xtka_4scjTAqx3Fn2Te1P_OB/lite/lite/data


In [ ]:
!ls

processed_data	process_ultrafine.py  release


In [ ]:
!python process_ultrafine.py

In [ ]:
%cd /content/drive/MyDrive/lite/lite

/content/drive/.shortcut-targets-by-id/156a1XWT2xtka_4scjTAqx3Fn2Te1P_OB/lite/lite


In [ ]:
!chmod +x lite.sh

In [ ]:
from torch import optim

In [ ]:
from torch.optim import AdamW

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python3 lite.py \
    --data_dir data/processed_data \
    --output_dir output \
    --train_batch_size 8 \
    --num_train_epochs 10 \
    --margin 0.1 \
    --save_epochs 10 \
    --learning_rate 1e-5 \
    --lamb 0.05

2025-04-30 18:38:31.410251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746038311.429544    8680 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746038311.435351    8680 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-30 18:38:31.454565: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassifi

In [ ]:
%cd "/content/drive/MyDrive/lite/lite/output"

/content/drive/.shortcut-targets-by-id/156a1XWT2xtka_4scjTAqx3Fn2Te1P_OB/lite/lite/output


In [ ]:
!ls

18_37_52_Apr_30_2025_batch4_margin0.1_lr1e-05lambda0.05
18_38_34_Apr_30_2025_batch8_margin0.1_lr1e-05lambda0.05


In [ ]:
%cd "/content/drive/MyDrive/lite/lite"

/content/drive/.shortcut-targets-by-id/156a1XWT2xtka_4scjTAqx3Fn2Te1P_OB/lite/lite


In [ ]:
import torch
from transformers import AutoTokenizer
from model import roberta_mnli_typing

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")

model = roberta_mnli_typing()

chkpt_path = "/content/drive/MyDrive/lite/lite/output/18_38_34_Apr_30_2025_batch8_margin0.1_lr1e-05lambda0.05/epochs10/model"

chkpt = torch.load(chkpt_path, map_location=device)

new_state_dict = {k: v for k, v in chkpt.items() if "roberta_module" in k}

model.load_state_dict(new_state_dict, strict=False)

model.to(device)
model.eval()

print("Model loaded successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded successfully!


In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=1e-5)

epoch = 10
loss = 0.00255

checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
}

checkpoint_path = "/content/drive/MyDrive/lite/lite/output/18_38_34_Apr_30_2025_batch8_margin0.1_lr1e-05lambda0.05/epochs10/checkpoint.pth"
torch.save(checkpoint, checkpoint_path)

print(f"Checkpoint saved at {checkpoint_path}")


Checkpoint saved at /content/drive/MyDrive/lite/lite/output/18_38_34_Apr_30_2025_batch8_margin0.1_lr1e-05lambda0.05/epochs10/checkpoint.pth


In [ ]:
import torch

checkpoint = torch.load("/content/drive/MyDrive/lite/lite/output/18_38_34_Apr_30_2025_batch8_margin0.1_lr1e-05lambda0.05/epochs10/checkpoint.pth",
                        map_location=torch.device("cpu"))
print(type(checkpoint))
print(checkpoint.keys())

<class 'dict'>
dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss'])


In [ ]:
new_state_dict = {k: v for k, v in checkpoint['model_state_dict'].items()}
model.load_state_dict(new_state_dict, strict=False)

model.to(device)
model.eval()

roberta_mnli_typing(
  (roberta_module): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutp

In [ ]:
def load_type_vocab(type_vocab_file):
    with open(type_vocab_file, "r") as f:
        type_vocab = [line.strip() for line in f.readlines()]
    return type_vocab

type_vocab = load_type_vocab("/content/drive/MyDrive/lite/lite/data/processed_data/types.txt")

In [ ]:
def predict_entity_type(sentence, entity, top_k=15):
    res_buffer = {}

    for batch_id in range(0, len(type_vocab), 8):
        batch_types = type_vocab[batch_id: batch_id + 8]
        inputs = [f"{sentence} {2 * tokenizer.sep_token} {entity} is a {label}." for label in batch_types]

        tokenized = tokenizer(inputs, padding=True, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**tokenized)[:, -1]

        confidences = outputs.detach().cpu().numpy().tolist()

        for idx in range(len(batch_types)):
            res_buffer[batch_types[idx]] = confidences[idx]

    confidence_ranking = sorted(res_buffer.items(), key=lambda item: -item[1])

    return confidence_ranking[:top_k]

In [ ]:
sentence = "Elon Musk founded SpaceX in 2002."
entity = "Elon Musk"

predicted_types = predict_entity_type(sentence, entity)
print("Predicted Types:", predicted_types)

Predicted Types: [('founder', 0.980595052242279), ('person', 0.9770424365997314), ('entity', 0.9678843021392822), ('founding', 0.9675833582878113), ('thing', 0.9666338562965393), ('individual', 0.9649014472961426), ('creator', 0.9621492624282837), ('full name', 0.9539445638656616), ('male', 0.9462495446205139), ('founders', 0.9447566866874695), ('name', 0.9440856575965881), ('entrepreneur', 0.9423936009407043), ('company', 0.9422285556793213), ('guy', 0.9399961233139038), ('businessperson', 0.934005081653595)]


In [ ]:
sentence1 = "The trophy would not fit the brown suitcase because it was too big."
entity1 = "it"
print("Predicted Types:", predict_entity_type(sentence1, entity1))

Predicted Types: [('trophy', 0.973510205745697), ('object', 0.9598625898361206), ('item', 0.9468600153923035), ('physical quantity', 0.9379496574401855), ('prize', 0.9267684817314148), ('big', 0.9183502793312073), ('award', 0.9155170321464539), ('thing', 0.9153642058372498), ('brown', 0.9117656946182251), ('large', 0.9063191413879395), ('luggage', 0.8987575769424438), ('artifact', 0.8953670859336853), ('color', 0.8906051516532898), ('entity', 0.8825176954269409), ('colour', 0.871986985206604)]


In [ ]:
sentence1 = "sid sriram has a huge fan base for his voice.He sings mostly in south indian languages."
entity1 = "He"
print("Predicted Types:", predict_entity_type(sentence1, entity1))

Predicted Types: [('singer', 0.9681880474090576), ('vocalist', 0.959831953048706), ('following', 0.9562976956367493), ('fan base', 0.956138014793396), ('voice', 0.9443802833557129), ('singing voice', 0.94329833984375), ('vocal', 0.9303038120269775), ('customer base', 0.9273030757904053), ('guy', 0.9170577526092529), ('popularity', 0.9011110663414001), ('audience', 0.8976266980171204), ('singing', 0.8897523880004883), ('person', 0.8812310695648193), ('fan palm', 0.8780252933502197), ('base', 0.8775641918182373)]


In [ ]:
sentence1 = "Tom Daschle was the Senate majority leader until South Dakota ‘s 2004 matchup ."
entity1 = "Tom Daschle"
print("Predicted Types:", predict_entity_type(sentence1, entity1))

Predicted Types: [('lawmaker', 0.9774667620658875), ('legislator', 0.9771511554718018), ('senator', 0.9743304252624512), ('leader', 0.9723047614097595), ('person', 0.9701734781265259), ('full name', 0.9667361974716187), ('officeholder', 0.9604176878929138), ('politician', 0.9532866477966309), ('thing', 0.9522510170936584), ('individual', 0.9501299262046814), ('male', 0.9478211998939514), ('entity', 0.9473834037780762), ('name', 0.9468435645103455), ('surname', 0.9407368898391724), ('legislature', 0.9377229809761047)]


In [ ]:
input_path = "/content/drive/MyDrive/lite/lite/data/processed_data/test_processed.json"
output_path = "/content/drive/MyDrive/lite/lite/data/processed_data/test_processed_subset.json"
num_samples = 20

with open(input_path, "r") as infile:
    lines = infile.readlines()

subset_lines = lines[:num_samples]

with open(output_path, "w") as outfile:
    outfile.writelines(subset_lines)


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
model_dir="/content/drive/MyDrive/lite/lite/output/18_38_34_Apr_30_2025_batch8_margin0.1_lr1e-05lambda0.05/epochs10"
eval_data_path="/content/drive/MyDrive/lite/lite/data/processed_data/test_processed_subset.json"
type_vocab_file="/content/drive/MyDrive/lite/lite/data/processed_data/types.txt"
batch_size=8
device=0

!CUDA_VISIBLE_DEVICES=0 python3 eval.py \
    --model_dir $model_dir \
    --eval_data_path $eval_data_path \
    --type_vocab_file $type_vocab_file \
    --batch $batch_size

2025-05-01 03:14:16.896897: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746069256.916439    1949 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746069256.924673    1949 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).


In [ ]:
import json
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import Counter

file_path = "/content/drive/MyDrive/lite/lite/output/18_38_34_Apr_30_2025_batch8_margin0.1_lr1e-05lambda0.05/epochs10/Evaluation_test_processed_subset.json"
top_k_list = [1, 3, 5]

with open(file_path, "r") as f:
    data = [json.loads(line) for line in f.readlines()]

y_true_all = []
y_pred_all = []

top_k_hits = {k: 0 for k in top_k_list}
total = len(data)

for example in data:
    gold_labels = set(example["annotation"])
    ranked = example["confidence_ranking"]
    pred_labels_sorted = sorted(ranked.items(), key=lambda x: -x[1])
    predicted_labels = [label for label, _ in pred_labels_sorted]

    y_true_all.append(gold_labels)
    y_pred_all.append(set(predicted_labels[:len(gold_labels)]))

    for k in top_k_list:
        top_k_pred = set(predicted_labels[:k])
        if gold_labels & top_k_pred:
            top_k_hits[k] += 1

all_labels = sorted(list({label for gold in y_true_all for label in gold}))
label_to_idx = {label: i for i, label in enumerate(all_labels)}

def binarize(label_set):
    binary = [0] * len(all_labels)
    for label in label_set:
        if label in label_to_idx:
            binary[label_to_idx[label]] = 1
    return binary

y_true_bin = [binarize(gold) for gold in y_true_all]
y_pred_bin = [binarize(pred) for pred in y_pred_all]

precision = precision_score(y_true_bin, y_pred_bin, average="micro")
recall = recall_score(y_true_bin, y_pred_bin, average="micro")
f1 = f1_score(y_true_bin, y_pred_bin, average="micro")

print(f"🔍 Micro Precision: {precision:.4f}")
print(f"🔍 Micro Recall:    {recall:.4f}")
print(f"🔍 Micro F1 Score:  {f1:.4f}")
print("📊 Top-k Accuracy:")
for k in top_k_list:
    acc = top_k_hits[k] / total
    print(f"  • Top-{k}: {acc:.4f}")


🔍 Micro Precision: 0.8209
🔍 Micro Recall:    0.5000
🔍 Micro F1 Score:  0.6215
📊 Top-k Accuracy:
  • Top-1: 0.7000
  • Top-3: 0.9000
  • Top-5: 0.9000


In [ ]:
import json
from sklearn.metrics import precision_score, recall_score, f1_score

file_path = "/content/drive/MyDrive/lite/lite/output/18_38_34_Apr_30_2025_batch8_margin0.1_lr1e-05lambda0.05/epochs10/Evaluation_test_processed_subset.json"

with open(file_path, "r") as f:
    data = [json.loads(line) for line in f.readlines()]

y_true_all = []
y_pred_all = []

for example in data:
    gold_labels = set(example["annotation"])
    ranked = example["confidence_ranking"]
    pred_labels_sorted = sorted(ranked.items(), key=lambda x: -x[1])
    predicted_labels = [label for label, _ in pred_labels_sorted]

    y_true_all.append(gold_labels)
    y_pred_all.append(set(predicted_labels[:len(gold_labels)]))

all_labels = sorted(list({label for gold in y_true_all for label in gold}))
label_to_idx = {label: i for i, label in enumerate(all_labels)}

def binarize(label_set):
    binary = [0] * len(all_labels)
    for label in label_set:
        if label in label_to_idx:
            binary[label_to_idx[label]] = 1
    return binary

y_true_bin = [binarize(gold) for gold in y_true_all]
y_pred_bin = [binarize(pred) for pred in y_pred_all]

precision = precision_score(y_true_bin, y_pred_bin, average="micro")
recall = recall_score(y_true_bin, y_pred_bin, average="micro")
f1 = f1_score(y_true_bin, y_pred_bin, average="micro")

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")


Precision: 49.56
Recall: 40.5
F1 Score: 48.0
